In [ ]:
using QuadGK
using DelimitedFiles
using Trapz
using Plots
using Interpolations
using PhysicalConstants.CODATA2018: c_0
using SpecialFunctions: besselj1

In [ ]:
# Define cosmological parameters for DUSTGRAIN-pathfinder simulations
# as they appear in https://doi:10.1093/mnras/sty2465 for LCDM background
Ω_b = 0.0481
h = 0.6731
H0 = h*100 # Hubble constant in km/s/Mpc

In [ ]:
# Define input folders
inpath = "/home/alessandro/phd/scripts/Dustgrain_outs/"
pk_path = inpath*"Pknl/"

In [ ]:
# Selecting cosmology
cosmo = "lcdm"
method = "HM"

In [ ]:
# For lcdm
Ω_m = 0.31345
Ω_cdm = Ω_m - Ω_b
Ω_de = 1 - Ω_m

plot_label = "LCDM"

In [ ]:
# Load matter power spectrum from file (assuming it contains only P_kappa(ell) values)
Pknl = readdlm(pk_path*"$cosmo"*"_$method"*".txt",',')
k = readdlm(pk_path*"k_"*"$cosmo"*"_$method"*".txt",',')
z = readdlm(pk_path*"z_"*"$cosmo"*"_$method"*".txt",',')

In [ ]:
Pknl

In [ ]:
# Sources redshift
zs = 2.0

In [ ]:
# Speed of light in km/s
# c = c_0/1e3
c = 299792.458

In [ ]:
# Dimemensionless Hubble parameter
function E(z)
    return sqrt(Ω_m * (1 + z)^3 + Ω_de)
end

# Comoving raidal distance r(z)
function r(z)
    integrand(z_prime) = 1 / E(z_prime)
    result, _ = quadgk(integrand, 0.0, z)
    return (c / H0) * result
end

In [ ]:
# Top-hat window function
function W_th(ℓ, θ)
    return 2 * besselj1(ℓ * θ) / (ℓ * θ)
end

In [ ]:
Pknl = Pknl/h^3
k = k*h

In [ ]:
z = vec(z)
k = vec(k)

In [ ]:
# Pzk = interpolate(Pknl, BSpline(Quadratic(Line(OnCell()))))
Pzk = interpolate((z,k), Pknl, Gridded(Linear()))
# Pzk = CubicSplineInterpolation(zk_nodes, Pknl)

In [ ]:
# Smoothed convergence
function κ_sm(z, θ_sm)
    integrand(ℓ) = Pzk(z, (ℓ+0.5)/r(z)) * W_th(ℓ,θ_sm)^2 * ℓ
    result, _ = quadgk(integrand, 1.0, 10000)
    return 2 * π * result   
end

In [ ]:
κ_sm(5e-3, 10)

In [ ]:
# Lensing efficiency
function W(z)
    return 1.5 * Ω_m * (H0/c)^2 * (1+z) * r(z) * (1-(r(z)/r(zs)))
end

In [ ]:
W(2.6)

In [ ]:
# Moments functional
function C(θ_sm, t)
    integrand(z) = W(z)^t * κ_sm(z,θ_sm)^(t-1) / E(z) / r(z)^(2*(t-1))
    result, _ = quadgk(integrand, 5e-3 , zs)
    return (c/H0) * result
end

In [ ]:
# Smoothing radius range
θ_s = range(2,stop=20,length=50)
length(θ_s)

In [ ]:
# Functional for the second κ moment
C_2 = zeros(length(θ_s))

In [ ]:

for (i,θ) in enumerate(θ_s)
    C_2[i] = C(θ,2)
end

C_2

In [ ]:
# Second moment
Q2 = 1
κ_2 = Q2*C_2